In [1]:
from platform import python_version

print(python_version())

3.7.9


In [2]:
import os
os.chdir('../..')

In [4]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan_postLSTM.model import DATGAN
#from modules.datgan_no_att.model import DATGAN
import networkx as nx

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [5]:
tf.__version__

'1.15.5'

In [6]:
df = pd.read_csv('../data/Chicago/full.csv')

In [7]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,23.42579,2,2,0,2,6,1,66,1.0,6,FTE,9.333333
1,drive,7,OTHER,1.71259,2,2,0,2,6,1,66,1.0,6,FTE,12.083333
2,drive,7,HOME_OTHER,21.77887,2,2,0,2,6,1,66,1.0,6,FTE,15.500000
3,drive,7,SHOPPING,2.02603,2,2,0,2,6,1,66,1.0,6,FTE,17.500000
4,drive,7,SHOPPING,0.87691,2,2,0,2,6,1,66,1.0,6,FTE,18.250000


In [8]:
continuous_columns = ["distance", "age", "departure_time"]

# DATGAN FC with normal graph

In [10]:
# personalised graph
graph = nx.DiGraph()
graph.add_edges_from([("age", "license"),
                      ("age", "education_level"),
                      ("gender", "work_status"),
                      ("education_level", "work_status"),
                      ("work_status", "hh_income"),
                      ("education_level", "hh_income"),
                      ("hh_income", "hh_descr"),
                      ("hh_income", "hh_size"),
                      ("hh_size", "hh_vehicles"),
                      ("hh_size", "hh_bikes"),
                      ("work_status", "trip_purpose"),
                      ("trip_purpose", "departure_time"),
                      ("trip_purpose", "distance"),
                      ("travel_dow", "choice"),
                      ("distance", "choice"),
                      ("departure_time", "choice"),
                      ("hh_vehicles", "choice"),
                      ("hh_bikes", "choice"),
                      ("license", "choice"),
                      # Non necessary links
                      ("education_level", "hh_size"),
                      ("work_status", "hh_descr"),
                      ("work_status", "hh_size"),
                      ("hh_income", "hh_bikes"),
                      ("hh_income", "hh_vehicles"),
                      ("trip_purpose", "choice")])
output_folder = '../output/DATGAN_TEST/'

# DATGAN FC with sequential graph (as in TGAN)

In [ ]:
# Linear graph following dataset
graph = nx.DiGraph()
tmp = []
cols = df.columns
for i in range(len(cols)-1):
    tmp.append((cols[i],cols[i+1]))
graph.add_edges_from(tmp)

output_folder = '../output/DATGAN_SEQ/'

# DATGAN FC without any links

In [ ]:
# No links
graph = nx.DiGraph()
cols = df.columns
for i in range(len(cols)):
    graph.add_node(cols[i])
    
output_folder = '../output/DATGAN_NOLINKS/'

# Training different epochs

In [12]:
datgan = DATGAN(continuous_columns, max_epoch = 100, steps_per_epoch=2000, batch_size=200, output=output_folder, gpu=0)
res = datgan.fit(df, graph)
datgan.save("100_ep", force=True)

[0413 14:49:03 @input_source.py:222] Setting up the queue 'QueueInput_1/input_queue' for CPU prefetching ...
-> Creating cell for age (in-edges: 0)


ValueError: Variable gen/LSTM/go_age already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "D:\Users\glede\anaconda3\envs\py37\lib\site-packages\tensorflow_core\python\framework\ops.py", line 1748, in __init__
    self._traceback = tf_stack.extract_stack()
  File "D:\Users\glede\anaconda3\envs\py37\lib\site-packages\tensorflow_core\python\framework\ops.py", line 3426, in _create_op_internal
    op_def=op_def)
  File "D:\Users\glede\anaconda3\envs\py37\lib\site-packages\tensorflow_core\python\framework\ops.py", line 3357, in create_op
    attrs, op_def, compute_device)
  File "D:\Users\glede\anaconda3\envs\py37\lib\site-packages\tensorflow_core\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "D:\Users\glede\anaconda3\envs\py37\lib\site-packages\tensorflow_core\python\framework\op_def_library.py", line 794, in _apply_op_helper
    op_def=op_def)


In [ ]:
datgan = DATGAN(continuous_columns, max_epoch = 200, steps_per_epoch=2000, batch_size=200, output=output_folder, gpu=0)
res = datgan.fit(df, graph)
datgan.save("200_ep", force=True)

In [ ]:
datgan = DATGAN(continuous_columns, max_epoch = 500, steps_per_epoch=2000, batch_size=200, output=output_folder, gpu=0)
res = datgan.fit(df, graph)
datgan.save("500_ep", force=True)

In [ ]:
datgan = DATGAN(continuous_columns, max_epoch = 1000, steps_per_epoch=2000, batch_size=200, output=output_folder, gpu=0)
res = datgan.fit(df, graph)
datgan.save("1000_ep", force=True)